In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid, RandomizedSearchCV, ParameterSampler
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy import linspace, asanyarray , typecodes, isfinite
from math import sqrt
from random import sample 
from random import seed
import xgboost as xgb
import matplotlib.pyplot as plt
from numpy import corrcoef
import pickle
from numpy import isnan
from scipy.stats import norm , truncnorm

In [10]:
# drugsInfo  = './Screened_Compounds.csv'
# drugsInfo  = pd.read_csv(drugsInfo)

drugsData = './target.csv'
drugsData = pd.read_csv(drugsData, header = 0)

pairs = './IC50.csv'
pairs = pd.read_csv(pairs)

SNP  = './SNP_CNV.csv'
SNP  = pd.read_csv(SNP)

Progeny = './progeny11.csv'
Progeny= pd.read_csv(Progeny, header = 0)

In [11]:
SNP.rename(columns={'Unnamed: 0': 'Cell line ID'},inplace=True)
Progeny.rename(columns={'Unnamed: 0': 'Cell line ID'},inplace=True)
drugsData.rename(columns={'Unnamed: 0': 'Name'},inplace=True)

In [5]:
# drugsInfo= drugsInfo.loc[:,["Drug ID","Drug Name","Target Pathway"]]
# drugsData= pd.merge(drugsData , drugsInfo,left_on = "Name", right_on = "Drug Name")
# drugsData = drugsData.loc[:, drugsData.columns != "Name"]
# drugsData.reset_index(drop=True, inplace=True)

In [12]:
pairs


,Unnamed: 0,906794,753531,753532,753535,1290724,1240121,1290725,1240122,753538,...,1330981,908120,1303911,1240224,905943,1298144,930301,1480367,1299069,1298218
0,Erlotinib,0.290898,-1.863485,3.974344,NaN,NaN,NaN,NaN,NaN,NaN,...,2.166515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rapamycin,0.207925,-2.941649,-0.746047,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.429004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sunitinib,4.126633,0.912353,1.281997,NaN,NaN,NaN,NaN,NaN,NaN,...,0.949086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PHA-665752,2.874543,2.704708,2.836948,NaN,NaN,NaN,NaN,NaN,NaN,...,1.877969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MG-132,-1.696581,-1.199009,-0.645981,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.306872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,AZD6244,2.523067,-0.523344,-2.540723,1.534598,1.537955,-1.017225,NaN,-0.582287,0.537602,...,NaN,3.723791,NaN,NaN,1.424759,-0.781229,2.488277,0.182523,NaN,-1.560211
261,Bicalutamide,4.588672,4.420135,4.433918,4.709197,4.546272,4.202429,NaN,4.903007,4.209484,...,NaN,5.256139,NaN,NaN,4.129703,5.111652,NaN,5.023501,NaN,3.468045
262,RDEA119 (rescreen),1.666861,NaN,-1.398112,1.784932,1.749408,-0.646167,NaN,-0.041710,0.141557,...,NaN,3.065830,NaN,NaN,1.331341,-0.263884,NaN,-0.257045,NaN,-0.386533
263,GDC0941 (rescreen),3.865132,NaN,-1.662177,0.525159,-0.439327,-0.414728,NaN,-0.423904,1.687893,...,NaN,1.103136,NaN,NaN,0.760399,0.771657,NaN,0.107793,NaN,1.150848


In [13]:
new_pairs = {}
new_pairs['drug']=[]
new_pairs['Cell line ID']=[]
new_pairs['IC50']=[]
'''repair pairs'''
cell_lines = pairs.iloc[0].index[1:].values
for i in range(len(pairs)):
    row = pairs.iloc[i]
    drug = row['Unnamed: 0']
    for cl in cell_lines:
        cell_line_ID = int(cl)
        IC_50 = row[cl]
        if pd.isnull(IC_50) == False:
            new_pairs['drug'].append(drug)
            new_pairs['Cell line ID'].append(cell_line_ID)
            new_pairs['IC50'].append(IC_50)

        

In [14]:
pairs = pd.DataFrame.from_dict(new_pairs)

In [15]:
'''First setup: seperate model for each drug - only cell line data- Progeny'''
all_data = pd.merge(pairs, Progeny, left_on = 'Cell line ID', right_on = 'Cell line ID')
cell_line_features = all_data.drop(['drug','Cell line ID',"IC50"], axis =1)
cell_line_features.columns

Index(['EGFR', 'Hypoxia', 'JAK.STAT', 'MAPK', 'NFkB', 'PI3K', 'TGFb', 'TNFa',
       'Trail', 'VEGF', 'p53'],
      dtype='object')

In [16]:
seed(7)
random_states = sample(range(100), 5)
random_states

[41, 19, 50, 83, 6]

In [17]:
drugs = all_data.drug.unique()

results = {}
params = {}


lower, upper = 0, 10
mu, sigma = 0.3, 0.2

rejects = []
num = 1
for drug in drugs:
    dicti = {}
    
    glob = True
    
    sample1 = all_data[all_data['drug']== drug]
    sample1 = sample1.reset_index(drop = True)
    cell_line_features = sample1.drop(['drug','Cell line ID',"IC50"], axis =1)

    X = cell_line_features
    Y = sample1['IC50']

    sum_RMSE_test = 0
    sum_RMSE_train = 0
    sum_corr_test = 0
    sum_corr_train = 0

    seeds = [80, 74, 7, 73, 50]

    i = 0
    for random_state in random_states :
        flag = True
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state= random_state)

        # Instantiate the model
        xgboost = xgb.XGBRegressor(seed = seeds[i], objective = 'reg:squarederror', n_jobs = 4, booster = 'gbtree')
        # Hyperparamter tuning on trainig set
        param_test1 = {'max_depth':[2,3,4,5,6,7],#8],
                   'subsample':[0.5,1],
                   'learning_rate':truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma),
                   'n_estimators' : [25,50 ,75, 101,100,102],
                   'reg_lambda' :[0., 0.5,1., 5]
                }

#         fit_params={"early_stopping_rounds":10, 
#             "eval_metric" : "rmse", 
#             "eval_set" : [(X_test, y_test)],
#                    "verbose" : False}


        # Instatiate grid search object
        grid = RandomizedSearchCV(estimator = xgboost, param_distributions = param_test1,
                            scoring="neg_mean_squared_error",
                            cv=5,
                            n_jobs = 4,
                            refit=True,
                            n_iter = 50,
                            random_state = seeds[i])

#Train the grid (compute cross validation for 50 parameter combinations)
        try:
            grid.fit(X_train, y_train)
        except ValueError:
            i+= 1
            flag = False
            glob = False
            print('upss  lek: ' + str(drug)+' random state: ' + str(random_state)+ ' seed: '+str(seeds[i])
                 + 'proba nr: ' +str(num))
            rejects.append(drug)

        if flag:
            d = {}
            d['params'] = grid.best_params_
            d['cv_results'] = grid.cv_results_
            d['var_imp'] =  grid.best_estimator_.feature_importances_

            dicti[random_state] = d

            best_model_test_predictions = grid.predict(X_test)
            best_model_train_predictions = grid.predict(X_train)

            #y_test
            RMSE_test = sqrt(mean_squared_error(y_test, best_model_test_predictions))
            corr_test = corrcoef(y_test, best_model_test_predictions)[0][1]
            if isnan(corr_test)==True:
                corr_test = 0

            #y_train
            RMSE_train = sqrt(mean_squared_error(y_train, best_model_train_predictions))
            corr_train = corrcoef(y_train , best_model_train_predictions)[0][1]
            if isnan(corr_train) == True:
                corr_train = 0

            sum_RMSE_test += RMSE_test
            sum_RMSE_train += RMSE_train
            sum_corr_train += corr_train
            sum_corr_test += corr_test

            print(random_state, seeds[i], grid.best_params_,  corr_train, corr_test)

            i+=1
    if glob:
        RMSE_test = sum_RMSE_test/5
        RMSE_train = sum_RMSE_train/5
        corr_test = sum_corr_test/5
        corr_train = sum_corr_train/5
        results[drug] = [drug, RMSE_train, RMSE_test, corr_train, corr_test]

        params[drug] = dicti

        print(grid.best_params_, )
        print(drug , num, '!!!!')

    num+=1
results = pd.DataFrame.from_dict(results, orient='index',
                            columns=['name','RMSE_train', 'RMSE_test', 'corr_train', 'corr_test'])


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


41 80 {'learning_rate': 0.13103297034123076, 'max_depth': 2, 'n_estimators': 102, 'reg_lambda': 0.0, 'subsample': 0.5} 0.8788748897791864 0.21280639449654395


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


19 74 {'learning_rate': 0.026608261622328988, 'max_depth': 3, 'n_estimators': 100, 'reg_lambda': 0.0, 'subsample': 0.5} 0.7969772200128262 0.29820084200465957


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


50 7 {'learning_rate': 0.0821461164419196, 'max_depth': 3, 'n_estimators': 101, 'reg_lambda': 5, 'subsample': 1} 0.7045097731699173 0.09500221449403594


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


83 73 {'learning_rate': 0.11271471446528816, 'max_depth': 2, 'n_estimators': 101, 'reg_lambda': 0.5, 'subsample': 1} 0.650568745779038 0.051640602506568274
6 50 {'learning_rate': 0.2171287495226979, 'max_depth': 2, 'n_estimators': 25, 'reg_lambda': 5, 'subsample': 1} 0.5062614898289729 0.13541527216002183
{'learning_rate': 0.2171287495226979, 'max_depth': 2, 'n_estimators': 25, 'reg_lambda': 5, 'subsample': 1}
PF-4708671 256 !!!!


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [18]:
results.describe()

,RMSE_train,RMSE_test,corr_train,corr_test
count,256.000000,256.000000,256.000000,256.000000
mean,1.045864,1.309743,0.692221,0.292034
std,0.336224,0.390716,0.066490,0.132847
min,0.375338,0.497863,0.355634,-0.034303
25%,0.786846,1.050242,0.649331,0.186347
50%,0.991638,1.246541,0.691828,0.293107
75%,1.262132,1.574508,0.735338,0.397181
max,2.381918,2.872356,0.931871,0.632764


In [19]:
results.to_csv('./prediction_per_drug_progeny_XGBoost.csv')

In [20]:
with open('params_per_drug_progeny_XGBoost.p', 'wb') as fp:
    pickle.dump(params, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
'''Second setup - per cell line- only drug data'''
all_data = pd.merge(pairs, drugsData, left_on = 'drug', right_on = 'Name')
cell_line_features = all_data.drop(['drug','Cell line ID',"IC50","Name",], axis =1)
cell_line_features.columns

Index(['EGFR', 'MTORC1', 'PDGFR', 'KIT', 'VEGFR', 'FLT3', 'RET', 'CSF1R',
       'MET', 'Proteasome',
       ...
       'ESR1', 'EGLN1', 'RNA helicase A', 'Induces reactive oxygen species',
       'NAMPT', 'TRAIL receptor agonist', 'L3MBTL3', 'DOT1L',
       'DNA alkylating agent', 'NAE'],
      dtype='object', length=228)

In [22]:
all_data

,drug,Cell line ID,IC50,Name,EGFR,MTORC1,PDGFR,KIT,VEGFR,FLT3,...,ESR1,EGLN1,RNA helicase A,Induces reactive oxygen species,NAMPT,TRAIL receptor agonist,L3MBTL3,DOT1L,DNA alkylating agent,NAE
0,Erlotinib,906794,0.290898,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Erlotinib,753531,-1.863485,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Erlotinib,753532,3.974344,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Erlotinib,753583,1.385394,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Erlotinib,910911,3.857275,Erlotinib,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227996,MLN4924,1298160,-0.240773,MLN4924,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
227997,MLN4924,908120,1.616776,MLN4924,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
227998,MLN4924,905943,0.130265,MLN4924,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
227999,MLN4924,1298144,0.497722,MLN4924,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
cells = all_data['Cell line ID'].unique()

results = {}
params = {}


lower, upper = 0, 10
mu, sigma = 0.3, 0.2

rejects = []
num = 1
for cl in cells:
    dicti = {}
    
    glob = True
    
    sample1 = all_data[all_data['Cell line ID']== cl]
    sample1 = sample1.reset_index(drop = True)
    cell_line_features = sample1.drop(['drug','Cell line ID',"IC50","Name"], axis =1)

    X = cell_line_features
    Y = sample1['IC50']

    sum_RMSE_test = 0
    sum_RMSE_train = 0
    sum_corr_test = 0
    sum_corr_train = 0

    seeds = [80, 74, 7, 73, 50]

    i = 0
    for random_state in random_states :
        flag = True
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state= random_state)

        # Instantiate the model
        xgboost = xgb.XGBRegressor(seed = seeds[i], objective = 'reg:squarederror', n_jobs = 4, booster = 'gbtree')
        # Hyperparamter tuning on trainig set
        param_test1 = {'max_depth':[2,3,4,5,6,7],#8],
                   'subsample':[0.5,1],
                   'learning_rate':truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma),
                   'n_estimators' : [25,50 ,75, 101,100,102],
                   'reg_lambda' :[0., 0.5,1., 5]
                }

#         fit_params={"early_stopping_rounds":10, 
#             "eval_metric" : "rmse", 
#             "eval_set" : [(X_test, y_test)],
#                    "verbose" : False}


        # Instatiate grid search object
        grid = RandomizedSearchCV(estimator = xgboost, param_distributions = param_test1,
                            scoring="neg_mean_squared_error",
                            cv=5,
                            n_jobs = 4,
                            refit=True,
                            n_iter = 50,
                            random_state = seeds[i])

#Train the grid (compute cross validation for 50 parameter combinations)
        try:
            grid.fit(X_train, y_train)
        except ValueError:
            i+= 1
            flag = False
            glob = False
            print('upss  lek: ' + str(cl)+' random state: ' + str(random_state)+ ' seed: '+str(seeds[i])
                 + 'proba nr: ' +str(num))
            rejects.append(cl)

        if flag:
            d = {}
            d['params'] = grid.best_params_
            d['cv_results'] = grid.cv_results_
            d['var_imp'] =  grid.best_estimator_.feature_importances_

            dicti[random_state] = d

            best_model_test_predictions = grid.predict(X_test)
            best_model_train_predictions = grid.predict(X_train)

            #y_test
            RMSE_test = sqrt(mean_squared_error(y_test, best_model_test_predictions))
            corr_test = corrcoef(y_test, best_model_test_predictions)[0][1]
            if isnan(corr_test)==True:
                corr_test = 0

            #y_train
            RMSE_train = sqrt(mean_squared_error(y_train, best_model_train_predictions))
            corr_train = corrcoef(y_train , best_model_train_predictions)[0][1]
            if isnan(corr_train) == True:
                corr_train = 0

            sum_RMSE_test += RMSE_test
            sum_RMSE_train += RMSE_train
            sum_corr_train += corr_train
            sum_corr_test += corr_test

            print(random_state, seeds[i], grid.best_params_,  corr_train, corr_test)

            i+=1
    if glob:
        RMSE_test = sum_RMSE_test/5
        RMSE_train = sum_RMSE_train/5
        corr_test = sum_corr_test/5
        corr_train = sum_corr_train/5
        results[cl] = [cl, RMSE_train, RMSE_test, corr_train, corr_test]

        params[cl] = dicti

        print(grid.best_params_, )
        print(drug , num, '!!!!')

    num+=1
results = pd.DataFrame.from_dict(results, orient='index',
                            columns=['name','RMSE_train', 'RMSE_test', 'corr_train', 'corr_test'])

/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


41 80 {'learning_rate': 0.17414214519595472, 'max_depth': 2, 'n_estimators': 100, 'reg_lambda': 0.5, 'subsample': 0.5} 0.814485580472336 0.4690740435134879


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


19 74 {'learning_rate': 0.27973931805974356, 'max_depth': 2, 'n_estimators': 100, 'reg_lambda': 0.5, 'subsample': 1} 0.8803202162696522 0.49482574038624527


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


50 7 {'learning_rate': 0.1261735388428072, 'max_depth': 7, 'n_estimators': 25, 'reg_lambda': 0.5, 'subsample': 1} 0.7878558667727329 0.5051840409399407


/home/ewa/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/ewa/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


83 73 {'learning_rate': 0.13383576522066204, 'max_depth': 2, 'n_estimators': 102, 'reg_lambda': 0.0, 'subsample': 1} 0.8451370001539646 0.17292224414224874


KeyboardInterrupt: 

In [24]:
results = pd.DataFrame.from_dict(results, orient='index',
                            columns=['name','RMSE_train', 'RMSE_test', 'corr_train', 'corr_test'])

In [25]:
results.to_csv('./prediction_per_cell_XGBoost.csv')

In [26]:
with open('params_per_cell_XGBoost.p', 'wb') as fp:
    pickle.dump(params, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
results.describe()

,name,RMSE_train,RMSE_test,corr_train,corr_test
count,4.950000e+02,495.000000,495.000000,495.000000,495.000000
mean,9.570464e+05,1.536616,2.311971,0.831036,0.414870
std,1.947319e+05,0.213614,0.217438,0.048333,0.092754
min,6.836650e+05,0.118967,1.668786,0.644948,-0.016336
25%,9.067970e+05,1.402288,2.163141,0.803591,0.358793
50%,9.092510e+05,1.527730,2.295239,0.836903,0.431787
75%,9.491535e+05,1.658308,2.452554,0.865478,0.481007
max,1.503371e+06,2.203657,3.026359,0.997572,0.636940


In [28]:
all_data = pd.merge(pairs, drugsData, left_on = 'drug', right_on = 'Name')
all_data=pd.merge(all_data, Progeny, left_on = 'Cell line ID', right_on = 'Cell line ID')
cell_line_features = all_data.drop(['drug','Cell line ID',"IC50","Name"], axis =1)
drugs = all_data.drug.unique()
cell_line_features.columns

Index(['EGFR_x', 'MTORC1', 'PDGFR', 'KIT', 'VEGFR', 'FLT3', 'RET', 'CSF1R',
       'MET', 'Proteasome',
       ...
       'Hypoxia', 'JAK.STAT', 'MAPK', 'NFkB', 'PI3K', 'TGFb', 'TNFa', 'Trail',
       'VEGF', 'p53'],
      dtype='object', length=239)

In [ ]:
results = []
sets = []
params = {}

lower, upper = 0, 10
mu, sigma = 0.3, 0.2

seeds = [80, 74, 7, 73, 50]

i = 0
drugs = all_data.drug.unique()
for random_state in random_states:
    
    dicti = {}

    cells_in_pairs = all_data["Cell line ID"].unique()#921 unique cell lines

    seed(random_state)
    Validation_set = sample(set(cells_in_pairs), k =130)
    cells_in_pairs = [x  for x in cells_in_pairs if x not in Validation_set]
    Validation_set = all_data[all_data['Cell line ID'].isin(Validation_set)]

    seed(random_state)
    Test_set = sample(set(cells_in_pairs), k= 131)
    Train_set = [x for x in cells_in_pairs if x not in Test_set]
    Test_set = all_data[all_data['Cell line ID'].isin(Test_set)]
    Train_set = all_data[all_data['Cell line ID'].isin(Train_set)]

    y_val = Validation_set["IC50"]
    y_test = Test_set["IC50"]
    y_train = Train_set["IC50"]

    minimum = min(y_train)
    maximum = max(y_train)

    y_train = [(x-minimum)/(maximum - minimum) for x in y_train]
    y_test = [(x-minimum)/(maximum-minimum) for x in y_test]
    y_val = [(x-minimum)/(maximum-minimum) for x in y_val]

    X_val = Validation_set.drop(['drug','Cell line ID',"IC50","Name"], axis = 1)
    X_test = Test_set.drop(['drug','Cell line ID',"IC50","Name"], axis = 1)
    X_train = Train_set.drop(['drug','Cell line ID',"IC50","Name"], axis = 1)

    Train_set['IC50'] = y_train
    Test_set['IC50'] = y_test
    Validation_set['IC50'] = y_val

    print(min(y_train),max(y_train),min(y_test),max(y_test),min(y_val),max(y_val))



    param_test1 = {'max_depth':range(2,11),
                   'subsample':[0.5,1],
                   'learning_rate': truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma),
                   'n_estimators' : [25,50 ,75, 101,100,102],
                   'reg_lambda' :[0., 0.5,1., 5]
                }
    parameters = ParameterSampler(param_test1, random_state = seeds[i], n_iter = 50)
    RMSE = 100
    for param in parameters:
        #initiate the model
        xgboost = xgb.XGBRegressor(seed = seeds[i], objective = 'reg:squarederror',
                                  max_depth = param['max_depth'],
                                  subsample = param['subsample'],
                                  learning_rate=param['learning_rate'],
                                  n_estimators = param['n_estimators'],
                                  reg_lambda = param['reg_lambda'])


        xgboost.fit(X_train, y_train)#, early_stopping_rounds=10, 
    #              eval_metric = "rmse", 
    #              eval_set = [(X_val, y_val)],
    #              verbose= False) 


        y_pred = xgboost.predict(X_val)

        if sqrt(mean_squared_error(y_val, y_pred)) < RMSE:
            RMSE = sqrt(mean_squared_error(y_val, y_pred))
            max_depth = param['max_depth']
            subsample = param['subsample']
            learning_rate=param['learning_rate']
            n_estimators = param['n_estimators']
            reg_lambda = param['reg_lambda']

    #initiate the model
    xgboost = xgb.XGBRegressor(seed= seeds[i], objective = 'reg:squarederror',
            max_depth = max_depth,
            subsample = subsample,
            learning_rate=learning_rate,
            n_estimators = n_estimators,
            reg_lambda = reg_lambda)



    #new train set containing Train_set and Validation_set
    Train_set = pd.concat([Train_set, Validation_set])
    y_train = Train_set["IC50"]
    X_train = Train_set.drop(['drug','Cell line ID',"IC50","Name"], axis = 1)

    #fit model
    xgboost.fit(X_train,y_train)#, early_stopping_rounds=10, 
    #             eval_metric = "rmse", 
    #             eval_set = [(X_test, y_test)],
    #             verbose= False) 

    #prediction
    y_pred_test = xgboost.predict(X_test)
    y_pred_train = xgboost.predict(X_train)

    Train_set['IC50_pred'] = y_pred_train
    Test_set['IC50_pred'] = y_pred_test
    drugs = all_data.DRUG_ID.unique()

    sets.append((Train_set, Test_set))

    dicti['params'] = {'max_depth' : max_depth,
            'subsample' : subsample,
            'learning_rate':learning_rate,
            'n_estimators' : n_estimators,
           'reg_lambda' : reg_lambda}

    dicti['var_imp'] =  xgboost.feature_importances_

    params[random_state] = dicti

    predictions = {}
    for drug in drugs:
        train_sample = Train_set[Train_set['drug']== drug]
        test_sample = Test_set[Test_set['drug']== drug]

        RMSE_test = sqrt(mean_squared_error(test_sample['IC50'], test_sample["IC50_pred"]))
        RMSE_train = sqrt(mean_squared_error(train_sample['IC50'], train_sample["IC50_pred"]))

        corr_test =  corrcoef(test_sample['IC50'], test_sample["IC50_pred"])[0][1]
        if isnan(corr_test) == True:
            corr_test = 0

        corr_train =  corrcoef(train_sample['IC50'], train_sample["IC50_pred"])[0][1]
        if isnan(corr_train) == True:
            corr_train = 0
        predictions[drug] = [drug, RMSE_test,RMSE_train , corr_test, corr_train , random_state]
    results.append(predictions)
    i+=1
    print(i, 'DONE')

/home/ewa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ewa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ewa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

0.0 1.0 0.05544415788603897 1.0507037959000447 -0.006734294610314879 1.0188245809252303
